In [1]:
#%pip install beautifulsoup4
#%pip install pip-system-certs

In [6]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import stardog
import os
import io
from langchain_community.graphs import Neo4jGraph
from PyPDF2 import PdfReader
from langdetect import detect
import spacy
import re
from unidecode import unidecode

In [7]:
from neo4j import GraphDatabase

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

- Buscar os links para a tese no Neo4j
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

Buscar links para a tese no Neo4j

Conectando ao Neo4j 

In [8]:

# Neo4j variables
#NEO4J_URL = os.getenv("NEO4J_URL")
#NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
#NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

#Connecting to the graph
#graph = Neo4jGraph(
#    url=NEO4J_URL,
#    username=NEO4J_USERNAME,
#    password=NEO4J_PASSWORD
#)

In [9]:
#Cypher query to get all theses from USP
query1 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:USP'})
Match (e:Exerpt)
RETURN distinct(t["uri"]) as Thesis, t["repository"] as repository 
"""

query2 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:USP'})
WHERE NOT EXISTS {(:Exerpt)--(t)}
RETURN distinct(t["uri"]) as Thesis, t["repository"] as repository 
"""

graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
results = graph.execute_query(query2).records
graph.close()

#results = graph.query(query2)
# Convert the results to a DataFrame
thesis_repository = pd.DataFrame(results, columns=["Thesis", "repository"])
thesis_repository = thesis_repository[200:300].reset_index(drop=True) 
thesis_repository

,Thesis,repository
0,tag:stardog:api:a_floresta_em_rede_a_informaca...,https://www.teses.usp.br/teses/disponiveis/27/...
1,tag:stardog:api:as_possibilidades_educacionais...,https://www.teses.usp.br/teses/disponiveis/103...
2,tag:stardog:api:a_instituicao_do_nome_analise_...,https://www.teses.usp.br/teses/disponiveis/8/8...
3,tag:stardog:api:vigilancia_epidemiologica_da_d...,https://www.teses.usp.br/teses/disponiveis/6/6...
4,tag:stardog:api:circular_e_preciso_a_expansao_...,https://www.teses.usp.br/teses/disponiveis/8/8...
...,...,...
95,tag:stardog:api:fonologia_e_morfossintaxe_da_l...,https://www.teses.usp.br/teses/disponiveis/8/8...
96,tag:stardog:api:os_indios_em_raizes_do_brasil_...,http://www.teses.usp.br/teses/disponiveis/8/81...
97,tag:stardog:api:entre_liberdade_e_cultura_mult...,https://www.teses.usp.br/teses/disponiveis/2/2...
98,tag:stardog:api:estudo_da_biodegradacao_bacter...,http://www.teses.usp.br/teses/disponiveis/97/9...


Criando as utils functions

In [12]:

# Funcao para coletar link do pdf

def get_pdf_link(url):
    
    #Fazer requisição e parsear o arquivo html
    f = requests.get(url, verify=True).text#, verify=False) 
    soup = bs(f, "html.parser")
    
    #Coletando link para as teses
    for doc in soup.find_all('a', href=True):
        if doc['href'].endswith('.pdf'):
            path = doc['href']
            prefix_uni = 'https://www.teses.usp.br'
            link = prefix_uni + path

            return link
    
    return None

#fazer download do arquivo
def download_file(pdf_link, thesis_uri):
    # NOTE the stream=True parameter below
    r = requests.get(pdf_link, verify=True, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open('tese2.pdf', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return 

#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))


# Função para extrair texto de uma tese em PDF e identificar entidades
def extratc_text(tese_path):
    
    # Processando o arquivo em PDF
    reader = PdfReader(tese_path)
    number_of_pages = len(reader.pages)
    excerpt = {}
    

    # Iterando por cada página
    for page_number in range(number_of_pages):
        
        
        # Extraindo o texto da página
        if page_number == 0:
            page_text = (reader.pages[page_number].extract_text() + '\n ' 
                        + reader.pages[page_number+1].extract_text()[:400])
        else:
            
            if page_number == number_of_pages -1:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text())
            
            else:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text() + '\n ' 
                            + reader.pages[page_number+1].extract_text()[:400])  
      
        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
            
        except:
            lang = 'Não_detectado'
        
        # Extraindo as entidades
        page_extracted = False
        if lang == 'pt':
            doc = nlp_pt(page_text)
            page_extracted = True
        if lang == 'en':
            doc = nlp_en(page_text)
            page_extracted = True
        
        if page_extracted:

            persons = []
            gpes = []
            #orgs = []
            for ent in doc.ents:
                if ent.label_ == "PER": #"PERSON":
                    persons.append(process_uri(ent.text))
                if ent.label_ == "LOC": #"GPE":
                    gpes.append(process_uri(ent.text))
                #if ent.label_ == "ORG":
                #    orgs.append(process_uri(ent.text))
                    
            excerpt[page_number] = {'text': page_text, 
                                    'lang': lang, 
                                    'persons': list(set(persons)), 
                                    'gpes': list(set(gpes)),
                                #'orgs': list(set(orgs))
                                }

    return excerpt

Iterando por todas as URI para extrair o texto e gravar no grafo NEO4j

In [13]:
# Gravar hora de início do processamento
start_time = pd.Timestamp.now()
parcial_time = pd.Timestamp.now()

for i in range(0,len(thesis_repository)):

    # Coletando o link do PDF
    url = thesis_repository['repository'][i]
    thesis_uri = thesis_repository['Thesis'][i][16:]
    pdf_link = get_pdf_link(url)
    if pdf_link is not None:
        print(f"{i} - Downloading and extracting: {thesis_uri} from {pdf_link}")
        # Download do arquivo PDF
        # Verificando se o arquivo já existe e removendo se necessário
        #if os.path.exists('tese.pdf'):
        #   os.remove('tese.pdf')
        download_file(pdf_link, thesis_uri)
        
        # Extraindo o texto e entidades
        tese_path = 'tese2.pdf'
        excerpt = extratc_text(tese_path)
        print(f"Extracted {thesis_uri}")
        
        # Salvando o triplas no grafo Neo4j
        for n in excerpt.keys():
     
            excerpt_uri = thesis_repository['Thesis'][i] + '_p_' + str(n)
            excerpt_text = excerpt[n]['text']
            if "'" in excerpt_text:
                excerpt_text = excerpt_text.replace("'", "")

            excerpt_page = n
            excerpt_lang = excerpt[n]['lang']
            #criando o dicionário para a inserção no grafo
            excerpt_dict = """{
                uri: \'""" + excerpt_uri +"""\',
                text: \'""" + excerpt_text + """\',
                page: \'""" + str(excerpt_page) + """\',
                lang: \'""" + excerpt_lang + """\'
                }"""
            
            #Criando a query para inserir o nó Exerpt

            query = """
            MATCH  (t:Thesis{uri: \'""" + thesis_repository['Thesis'][i] +  """\'})
            MERGE  (e:Exerpt""" + str(excerpt_dict) + """)-[r:BFO_0000050]-(t);

            """
            
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()
            #results = graph.query(query)
                
            #Criando as relações com as entidades
            for person in excerpt[n]['persons']:
                query = """
                MATCH  (e:Exerpt{uri: '""" + excerpt_uri + """'})
                MERGE  (p:Person{uri: \'tag:stardog:api:""" + person + """\'})
                MERGE  (e)-[r:mentions]-(p);

                """
                graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
                results = graph.execute_query(query)
                graph.close()
            #results = graph.query(query)

            for gpe in excerpt[n]['gpes']:
                query = """
                MATCH  (e:Exerpt{uri: '""" + excerpt_uri + """'})
                MERGE  (p:Place{uri: \'tag:stardog:api:""" + gpe + """\'})
                MERGE  (e)-[r:mentions]-(p);

                """
                graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
                results = graph.execute_query(query)
                graph.close()
                #results = graph.query(query)

        #graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        #results = graph.execute_query("""CALL apoc.cypher.runMany(' """ + query[:-1] + """ ', {})""")  # Remove the last semicolon
        #results = graph.execute_query(query)
        #graph.close()

        print(f"Processed and added to graph: {thesis_uri}")
        #Deletando o arquivo PDF após o processamento
        os.remove('tese2.pdf')
        print(f"Total time: {pd.Timestamp.now() - start_time}")
        print(f"Partial time: {pd.Timestamp.now() - parcial_time}")
        # Atualizando a hora parcial
        parcial_time = pd.Timestamp.now()
    else:
        print(f"No PDF link found for {thesis_uri}")

0 - Downloading and extracting: a_floresta_em_rede_a_informacao_como_instrumento_de_mediacao_e_resistencia_cultural from https://www.teses.usp.br/teses/disponiveis/27/27151/tde-04052022-115403/publico/WalaceSoaresdeOliveiracorrigida.pdf
Extracted a_floresta_em_rede_a_informacao_como_instrumento_de_mediacao_e_resistencia_cultural
Processed and added to graph: a_floresta_em_rede_a_informacao_como_instrumento_de_mediacao_e_resistencia_cultural
Total time: 0 days 00:02:37.223397
Partial time: 0 days 00:02:37.223397
1 - Downloading and extracting: as_possibilidades_educacionais_dos_acervos_digitais_indigenas_estudo_de_caso_a_partir_de_projetos_glamwiki from https://www.teses.usp.br/teses/disponiveis/103/103131/tde-06022023-171051/publico/LucianaConradoMartinORIGINAL.pdf
Extracted as_possibilidades_educacionais_dos_acervos_digitais_indigenas_estudo_de_caso_a_partir_de_projetos_glamwiki
Processed and added to graph: as_possibilidades_educacionais_dos_acervos_digitais_indigenas_estudo_de_caso_

KeyboardInterrupt: 

In [ ]:
### Ajustar o codigo cypher para inserir todas as triplas de uma vez só vez

In [22]:
print(query)

 
            MATCH  (t:Thesis{uri: 'tag:stardog:api:analise_da_ancestralidades_genetica_da_populacao_de_sao_paulo'})
            MERGE  (e:Exerpt{
                uri: 'tag:stardog:api:analise_da_ancestralidades_genetica_da_populacao_de_sao_paulo_p_0',
                text: 'UNIVERSIDADE DE SÃO PAULO  
INSTITUTO DE BIOCIÊNCIAS  
PROGRAMA INTERUNIDADES DE PÓS -GRADUAÇÃO EM BIOINFORMÁTICA  
RAP
HAEL BRUNO AMEMIYA  
A
nálise da ancestralidade genética da população de São Paulo  
S
ão Paulo  
2024 
  
 
 
 
  
 
 
  
 
 
   
 
   
 
 
 
 
 
 
 
 
  
 
 
  
 
   
 
 
   
 
  
 
 
 
 
 ',
                page: '0',
                lang: 'pt'
                })-[r:BFO_0000050]-(t);

            
                MATCH  (e:Exerpt{uri: 'tag:stardog:api:analise_da_ancestralidades_genetica_da_populacao_de_sao_paulo_p_0'})
                MERGE  (p:Person{uri: 'tag:stardog:api:hael'})
                MERGE  (e)-[r:mentions]-(p);
                
                MATCH  (e:Exerpt{uri: 'tag:stardog:a

Extrair texto do PDF

In [22]:
excerpt = extratc_text(thesis_uri + '.pdf')

In [23]:
thesis_repository['Thesis'][i]

'tag:stardog:api:o_jeito_yanomami_de_pendurar_redes'

Adicionar excerpt no Grafo Neo4j

In [40]:
#Cypher query 
for n in range(len(excerpt)):
     
    excerpt_uri = thesis_repository['Thesis'][i] + '_p_' + str(n)
    excerpt_text = excerpt[n]['text']
    excerpt_page = n
    excerpt_lang = excerpt[n]['lang']
    #criando o dicionário para a inserção no grafo
    excerpt_dict = """{
        uri: '""" + excerpt_uri +"""',
        text: '""" + excerpt_text + """',
        page: '""" + str(excerpt_page) + """',
        lang: '""" + excerpt_lang + """'
    }"""
    
    #Criando a query para inserir o nó Exerpt
    query = """
    MATCH  (t:Thesis{uri: '""" + thesis_repository['Thesis'][i] +  """'})
    MERGE  (e:Exerpt""" + str(excerpt_dict) + """)-[r:BFO_0000050]-(t)

    """

    results = graph.query(query)
        
    #Criando as relações com as entidades
    for person in excerpt[n]['persons']:
        query = """
        MATCH  (e:Exerpt{uri: '""" + excerpt_uri + """'})
        MERGE  (p:Person{uri: 'tag:stardog:api:""" + person + """'})
        MERGE  (e)-[r:mentions]-(p)
        """
        results = graph.query(query)

    for gpe in excerpt[n]['gpes']:
        query = """
        MATCH  (e:Exerpt{uri: '""" + excerpt_uri + """'})
        MERGE  (p:Place{uri: 'tag:stardog:api:""" + gpe + """'})
        MERGE  (e)-[r:mentions]-(p)
        """
        results = graph.query(query)



In [41]:
os.remove(thesis_uri + '.pdf')

In [39]:
query = """
        MATCH  (e:Exerpt{uri: '""" + excerpt_uri + """'})
        MERGE  (p:Person{uri: 'tag:stardog:api:""" + excerpt[263]['persons'][0] + """'})
        MERGE  (e)-[r:mentions]-(p)
        """
print (query)


        MATCH  (e:Exerpt{uri: 'tag:stardog:api:o_jeito_yanomami_de_pendurar_redes_p_263'})
        MERGE  (p:Person{uri: 'tag:stardog:api:wilbert'})
        MERGE  (e)-[r:mentions]-(p)
        


In [51]:
print(query)


MERGE  (e:Exerpt{
    uri: 'tag:stardog:api:revisao_narrativa_para_subsidiar_o_entendimento_do_panorama_da_cobertura_vacinal_em_criancas_menores_de_um_ano_de_idade_no_contexto_brasileiro_p_0',
    text: ' 
 UNIVERSIDADE DE SÃO PAULO  
FACULDADE DE MEDICINA  
 
 
 
 
 
SILVANETE MENDES MONTAGNINI  
 
 
 
 
REVISÃO NARRATIVA PAR A SUBSIDIAR O ENTEND IMENTO DO PANORAMA 
DA COBERTURA VACINAL  EM CRIANÇAS MENORES  DE UM ANO DE IDADE , 
NO CONTEXTO BRASILEI RO 
 
 
 
 
 
 
 
 
São Paulo  
2024  
  
 SILVANETE MENDES MONTAGNINI  
 
 
 
 
 
REVISÃO NARRATIVA PAR A SUBSIDIAR O ENTEND IMENTO DO PANORAMA 
DA COBERTURA VACINAL  EM CRIANÇAS MENORES  DE UM ANO DE IDADE , 
NO CONTEXTO BRASILEI RO 
 
 
 
Versão corrigida  
Resolução CoPGr 6018/11, de 01 de novembro de 2011 . 
A versão or iginal está disponível na Biblioteca FMUSP.  
 
 
 
Dissertação apresentada à Faculdade de 
Medicina da Universid',
    page: '0',
    lang: 'pt'
})-[r:BFO_0000050]-(t:Thesis{uri: 'tag:stardog:api:revisao_narrativa_p

In [46]:
excerpt[60]

{'text': '59 \n de ESAVIS vem se estabelecendo e ganhando maior dimensão, mas a autonomia na \nprodução de vacinas ainda é menor, em comparação à Índia . Fabricantes de países de \nbaixo e médio nível socioeconômico têm um papel cada vez maior no fornecimento de \nvacinas, e sua a entrada no mercado estimula quedas de preço das principais vacinas .  \n 60 \n 6. DISCUSSÃO  \n \nPelo exposto depreen de-se que, apesar do sucesso dos programas de vacinação, já \nhavia sinais de preocupação em relação à manutenção das conquistas obtidas. A queda \ndas coberturas vacinais vinha ocorrendo principalmente a partir de 2016.  \nA vacinação se caracteriza por interdep endência de responsabilidades individuais, \ncoletivas e institucionais. Esta dissertação priorizou a investigação sobre as \nresponsabilidades institucionais na queda das coberturas vacinais.  \nNesta revisão narrativa sobre a interrelação entre a cobertura vaci nal no Brasil e \noutros indicadores de qualidade do SUS foram encontra

In [ ]:
def extratc_text_NER(tese_uri, tese_path):

    triplas = """ """

    # Processando o arquivo em PDF
    reader = PdfReader(tese_path)
    number_of_pages = len(reader.pages)

    # Iterando por cada página
    for page_number in range(number_of_pages):
        

        # Extraindo o texto da página
        if page_number == 0:
            page_text = (reader.pages[page_number].extract_text() + '\n ' 
                        + reader.pages[page_number+1].extract_text()[:400])
        else:
            
            if page_number == number_of_pages -1:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text())
            
            else:
                page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                            + reader.pages[page_number].extract_text() + '\n ' 
                            + reader.pages[page_number+1].extract_text()[:400])  

        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
        except:
            lang = None

        # Extraindo as entidades
        if lang == 'pt':
            doc = nlp_pt(page_text)
        if lang == 'en':
            doc = nlp_en(page_text)

        persons = []
        gpes = []
        #orgs = []
        for ent in doc.ents:
            if ent.label_ == "PER": #"PERSON":
                persons.append(process_uri(ent.text))
            if ent.label_ == "LOC": #"GPE":
                gpes.append(process_uri(ent.text))
            #if ent.label_ == "ORG":
            #    orgs.append(process_uri(ent.text))


            
        tripla = """
        stardog:""" + tese_uri + "__page_" + str(page_number + 1) + """ rdf:type bibo:Excerpt.
        stardog:""" + tese_uri +  """ <http://purl.obolibrary.org/obo/BFO_0000051> stardog:""" + tese_uri + "__page_" + str(page_number + 1 ) + """.
        stardog:""" + tese_uri + "__page_" + str(page_number + 1 ) + """ bibo:pages '""" + str(page_number + 1) +  """'. 
        """
        if lang == 'pt':
            tripla = tripla + "stardog:" + tese_uri + "__page_" + str(page_number + 1 ) + " <http://purl.org/dc/terms/description> '" + page_text.replace("'","").replace(u'\\', u' ') +  "'@pt."
        if lang == 'en':
            tripla = tripla + "stardog:" + tese_uri + "__page_" + str(page_number + 1 ) + " <http://purl.org/dc/terms/description> '" + page_text.replace("'","").replace(u'\\', u' ') +  "'@en."
        
        #print(set(persons), set(gpes))
        for per in set(persons):  
            tripla = tripla + """
            stardog:""" + per + """ rdf:type foaf:Person.
            stardog:""" + tese_uri + "__page_" + str(page_number + 1) + """ <https://schema.org/mentions> stardog:""" + per + """. 
            """

        for gpe in set(gpes): 
            tripla = tripla + """
            stardog:""" + gpe + """ rdf:type <https://schema.org/Place>.
            stardog:""" + tese_uri + "__page_" + str(page_number + 1) + """ <https://schema.org/mentions> stardog:""" + gpe + """. 
            """

        triplas = triplas + " " + tripla

    add_triplas_to_stardog(prefixos, triplas)
    return

In [1]:
#!python -m spacy download pt_core_news_lg
#!python -m spacy download en_core_web_lg
#%pip install -U scikit-learn

In [1]:
import os
import io
import stardog
import pandas as pd
import spacy
from spacy import displacy
from sklearn.cluster import DBSCAN
import numpy as np
import requests
from unidecode import unidecode
import re
import json
from langdetect import detect

### Specify Stardog connection details